In [10]:
import pandas as pd
import os
import pickle

In [11]:
#Download the initial test statistics
df = pd.read_csv('Data/permutation_test_files_metabolomics/initial_tstats.csv', index_col=0)


In [12]:
path = os.getcwd() + '\\Data\\permutation_test_files_metabolomics'

In [48]:
final_list = []

for filename in os.listdir(path):
    if filename != 'initial_tstats.csv':
        file_num = int(filename[3:-4])
        if file_num < 6:
            with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
                list1 = pickle.load(f)
            final_list.append(list1)

In [49]:
len(final_list)

5

In [31]:
sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    for list1 in final_list:  #len(final_list) = number of permutations
        if abs(list1[index]) >= comparison:   
            counter += 1
    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

1316


### Another alternative: (but converting to numpy array actually seems to take up more memory)

In [44]:
import numpy as np

final_matrix = np.empty((0,len(df)), int)

for filename in os.listdir(path):
    if filename != 'initial_tstats.csv':
        with open(os.path.join(path, filename), 'rb') as f: # open in readonly mode
            list1 = pickle.load(f)
        final_matrix = np.append(final_matrix, np.array([list1]), axis=0)

In [36]:
final_matrix.shape

(10, 10296)

In [45]:
final_matrix = np.array(final_list) 

final_matrix.shape

sig_edges = []
edgelist = df.index

for index,pathway_pair in enumerate(edgelist):   #test all pathways
    comparison = df.Initial_tstat[index]    #get initial test statistic
    counter = 0
    
    col = abs(final_matrix[:,index])  #index a column and turn into absolute values
    counter = (col >= comparison).sum()

    
    p_val = (counter/len(final_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  
        
print(len(sig_edges))

1316


In [46]:
from sys import getsizeof
print(getsizeof(final_list))
print(getsizeof(final_matrix))

184
823808


In [35]:

with open ('metabolomics/Data/sig_edges.txt', 'w') as file:
     file.write(','.join(str(i) for i in sig_edges))

-0.5646745805638236


### Read in significant edges files and compare the results

In [66]:
with open('Data/permutation_test_files_metabolomics/sig_edges_10kpermutations.txt') as f:
    lines = f.readlines()

sig_list = lines[0].split(';')
print(len(sig_list))
print(sig_list[0])

268
R-HSA-1483257, R-HSA-1368082
